In [1]:
import pandas as pd
from random import seed
from random import randint
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
seed(1)

In [2]:
df1 = pd.read_csv('query_v1.csv') ## https://outschool-metabase.herokuapp.com/question/6319
df2 = pd.read_csv('query_v2.csv') ## https://outschool-metabase.herokuapp.com/question/6320

In [3]:
def convert_to_hours(date_time_str):
    date_time_obj = datetime.strptime(date_time_str[:19], '%Y-%m-%dT%H:%M:%S')
    return date_time_obj.strftime("%I:%M%p")

def convert_to_date(date_time_str):
    date_time_obj = datetime.strptime(date_time_str[:19], '%Y-%m-%dT%H:%M:%S')
    return date_time_obj.strftime("%Y-%m-%d")

def convert_to_week(date_time_str):
    date_time_obj = datetime.strptime(date_time_str[:19], '%Y-%m-%dT%H:%M:%S')
    date_time_obj_2 = date_time_obj.strftime("%Y-%m-%d")
    date_time_obj_3 = date_time_obj - timedelta(days=date_time_obj.weekday())
    return date_time_obj_3.strftime("%Y-%m-%d")


In [4]:
def find_schedule_conflicts(section_1, section_2, data):

    section_1_meetings = data[data['section_uid'] == section_1].sort_values(by=['meeting_start_time'])
    section_2_meetings = data[data['section_uid'] == section_2].sort_values(by=['meeting_start_time'])
    
    relevant_columns = ['title',
                        'section_uid',
                        'meeting_start_time',
                        'meeting_start_dow',
                        'meeting_start_hour_of_day',
                        'meeting_end_time'
                       ]
#     display(section_1_meetings[relevant_columns])
#     display(section_2_meetings[relevant_columns])
#     print('\n')
    
    schedule_conflicts = 0 
    
    for i in range(len(section_1_meetings)):
        row = section_1_meetings.iloc[i]
        s1_meeting_start_time = row['meeting_start_time']
        s1_meeting_end_time = row['meeting_end_time']
        s1_meeting_start_dow = row['meeting_start_dow']
        s1_meeting_start_hod = row['meeting_start_hour_of_day']
        
        for j in range(len(section_2_meetings)):
            row = section_2_meetings.iloc[j]
            s2_meeting_start_time = row['meeting_start_time']
            s2_meeting_end_time = row['meeting_end_time']
            s2_meeting_start_dow = row['meeting_start_dow']
            s2_meeting_start_hod = row['meeting_start_hour_of_day']
            
            if (s1_meeting_start_time <= s2_meeting_start_time <= s1_meeting_end_time) or (s1_meeting_start_time <= s2_meeting_end_time <= s1_meeting_end_time) or (s2_meeting_start_time <= s1_meeting_start_time <= s2_meeting_end_time) or (s2_meeting_start_time <= s1_meeting_end_time <= s2_meeting_end_time):
                schedule_conflicts += 1
#                 print('SCHEDULE CONFLICT')
#                 print('Section 1 meeting: ', s1_meeting_start_time, ' - ', s1_meeting_end_time)
#                 print('Section 2 meeting: ', s2_meeting_start_time, ' - ', s2_meeting_end_time)
    
    return schedule_conflicts
    

In [9]:
def build_schedule(learner_age, subjects, class_types, budget, teacher_min_star_rating, availability_dow, availability_hod):
    
    df1_f1 = df1[df1['subject'].isin(subjects)]
    df1_f2 = df1_f1[(df1_f1['age_min'] <= learner_age) & (df1_f1['age_max'] >= learner_age)] 
    df1_f3 = df1_f2[df1_f2['teacher_avg_star_rating'] >= teacher_min_star_rating]
    df1_f4 = df1_f3[df1_f3['class_type'].isin(class_types)]
    
    eligible_sections = []

    for section_uid in df1_f4['section_uid'].unique():
        eligible = 'true'
        meeting_dow = df1_f4[df1_f4['section_uid'] == section_uid]['meeting_start_dow']
        meeting_hod = df1_f4[df1_f4['section_uid'] == section_uid]['meeting_start_hour_of_day']

        l1 = list(set(meeting_dow) - set(availability_dow))
        l2 = list(set(meeting_hod) - set(availability_hod))

        if len(l1) == 0 and len(l2) == 0: 
            eligible_sections.append(section_uid)
    
    df2_f1 = df2[df2['subject'].isin(subjects)]
    df2_f2 = df2_f1[(df2_f1['age_min'] <= learner_age) & (df2_f1['age_max'] >= learner_age)] 
    df2_f3 = df2_f2[df2_f2['teacher_avg_star_rating'] >= teacher_min_star_rating]
    df2_f4 = df2_f3[df2_f3['section_uid'].isin(eligible_sections)]
    df2_f5 = df2_f4[df2_f4['class_type'].isin(class_types)]
    
    spend = 0
    purchased_sections = []
    visited_indices = []
    
    while spend <= budget or abs(budget - spend) < 20:
        
        random_index = randint(0, len(df2_f5) - 1)
        
        if random_index in visited_indices:
            pass 
        
        else:
            visited_indices.append(random_index)
            random_row = df2_f5.iloc[random_index]
            activity_title = random_row['title']
            section_uid = random_row['section_uid']
            price_dollars = random_row['price_dollars']
            
            
            if len(purchased_sections) == 0:
                purchased_sections.append(section_uid)
                spend += price_dollars
            
            else: 
                schedule_conflicts = 0
                for prev_section in purchased_sections:
                    schedule_conflicts += find_schedule_conflicts(prev_section, section_uid, df1_f4)
                
                if schedule_conflicts > 0:
                    pass
                else: 
                    purchased_sections.append(section_uid)
                    spend += price_dollars
    
    class_list = df2_f5[df2_f5['section_uid'].isin(purchased_sections)].copy()
    meeting_list = df1_f4[df1_f4['section_uid'].isin(purchased_sections)].sort_values(by='meeting_start_time').copy()
    
    class_list_cols = ['uid', 'title', 'summary', 'subject', 'class_type', 'price_dollars']
    class_list_filtered = class_list[class_list_cols].copy().reset_index(drop=True)
    class_list_filtered['link'] = 'http://www.outschool.com/classes/' + class_list_filtered['uid']
    class_list_filtered.drop(columns='uid', inplace=True)
    
    meeting_list_cols = ['title', 'meeting_start_week', 'meeting_start_date', 'meeting_start_dow', 'meeting_start_hour', 'meeting_end_hour']
    meeting_list['meeting_start_week'] = meeting_list['meeting_start_time'].apply(lambda x: convert_to_week(x))
    meeting_list['meeting_start_date'] = meeting_list['meeting_start_time'].apply(lambda x: convert_to_date(x))
    meeting_list['meeting_start_hour'] = meeting_list['meeting_start_time'].apply(lambda x: convert_to_hours(x))
    meeting_list['meeting_end_hour']   = meeting_list['meeting_end_time'].apply(lambda x: convert_to_hours(x))
    meeting_list_filtered = meeting_list[meeting_list_cols].copy().reset_index(drop=True)
    
#     print('\n')
#     print("Class list output")
    display(class_list_filtered)
    
#     print('\n')
#     print("Meeting list output")
    display(meeting_list_filtered)
    
#     with pd.ExcelWriter('sample_output.xlsx') as writer:  
#         class_list_filtered.to_excel(writer, sheet_name='class_list')
#         meeting_list_filtered.to_excel(writer, sheet_name='meeting_list')
    
    return class_list_filtered, meeting_list_filtered

In [10]:
learner_age = 10
subjects = set(['English', 'Social Studies', 'Arts', 'Science & Nature'])
class_types = set(['one_time', 'camp', 'short', 'semester', 'ongoing'])
budget = 200
hours_of_class = 30
start_date = '2020-07-12'
end_date = '2020-07-25'
availability_dow = set(['Mon', 'Tue', 'Sat', 'Sun'])
availability_hod = set([10, 11, 14, 15])
teacher_min_star_rating = 4.5

In [7]:
example_schedule = build_schedule(learner_age, 
                                  subjects, 
                                  class_types, 
                                  budget, 
                                  teacher_min_star_rating,
                                  availability_dow,
                                  availability_hod
                                 )


,title,summary,subject,class_type,price_dollars,link
0,My Hero Academia and the Real Science of Quirks,"In this class, we will look at the various her...",Science & Nature,short,26.0,http://www.outschool.com/classes/b4a6a564-3a25...
1,Dragonology,Dare To Dance With Dragons And Learn Their Lore!,English,short,35.0,http://www.outschool.com/classes/4467f390-5c5a...
2,Introduction to Essay Writing: Middle School B...,A 4-week writing class that teaches students t...,English,short,56.0,http://www.outschool.com/classes/63869671-c906...
3,Contemporary/Modern Dance-Beginners,In this dance class we will explore the fundam...,Arts,one_time,18.0,http://www.outschool.com/classes/1a8ce13d-bdaf...
4,Horses Around the World: Iceland,A one-time class where students learn about th...,Social Studies,one_time,18.0,http://www.outschool.com/classes/426b63b1-43c6...
5,Alexander Hamilton: The Original American Succ...,In this class we will learn about his incredib...,Social Studies,one_time,10.0,http://www.outschool.com/classes/9fabe8ac-ab45...
6,"Harry Potter - Astronomy, Astrology, and Greek...","Learn the basics of astronomy (stars, planets,...",Science & Nature,short,68.0,http://www.outschool.com/classes/def45290-0e4d...


,title,meeting_start_week,meeting_start_date,meeting_start_dow,meeting_start_hour,meeting_end_hour
0,My Hero Academia and the Real Science of Quirks,2020-07-13,2020-07-13,Mon,10:00AM,10:55AM
1,Contemporary/Modern Dance-Beginners,2020-07-13,2020-07-13,Mon,11:30AM,12:25PM
2,Horses Around the World: Iceland,2020-07-13,2020-07-13,Mon,03:00PM,03:45PM
3,Alexander Hamilton: The Original American Succ...,2020-07-13,2020-07-14,Tue,10:00AM,10:50AM
4,Introduction to Essay Writing: Middle School B...,2020-07-13,2020-07-14,Tue,11:00AM,11:45AM
5,"Harry Potter - Astronomy, Astrology, and Greek...",2020-07-13,2020-07-14,Tue,03:00PM,03:50PM
6,Dragonology,2020-07-13,2020-07-18,Sat,03:00PM,03:55PM
7,My Hero Academia and the Real Science of Quirks,2020-07-20,2020-07-20,Mon,10:00AM,10:55AM
8,Introduction to Essay Writing: Middle School B...,2020-07-20,2020-07-21,Tue,11:00AM,11:45AM
9,"Harry Potter - Astronomy, Astrology, and Greek...",2020-07-20,2020-07-21,Tue,03:00PM,03:50PM


In [12]:
with pd.ExcelWriter('sample_output_full.xlsx') as writer:
    for i in range(1,5):
        print('Example schedule #' + str(i) + ':')
        class_list, meeting_list = build_schedule(learner_age, 
                                      subjects, 
                                      class_types, 
                                      budget, 
                                      teacher_min_star_rating,
                                      availability_dow,
                                      availability_hod
                                     ) 
        class_list.to_excel(writer, sheet_name='class_list_' + str(i))
        meeting_list.to_excel(writer, sheet_name='meeting_list_' + str(i))
        print('___________________________________________________________________________________________________________')
        print('\n')

    

Example schedule #1:


,title,summary,subject,class_type,price_dollars,link
0,Famous People who were Productive during Quara...,This one-time class will present and discuss t...,Social Studies,one_time,10.0,http://www.outschool.com/classes/1acbde2a-d8d1...
1,Exploring Watercolor Paint Techniques,Learn a variety of techniques for using waterc...,Arts,one_time,12.0,http://www.outschool.com/classes/31b4c333-53e4...
2,Adventures in Crochet - Make a Potholder!,"In this three week class, students will learn ...",Arts,short,28.0,http://www.outschool.com/classes/a6c3aebc-735f...
3,Tree of Life: Sharks and Rays,"One of the two great branches of fish, sharks ...",Science & Nature,one_time,15.0,http://www.outschool.com/classes/518ef2a5-bfd0...
4,How To Make Awesome Animated Movies (Intro to ...,This course is an introduction to 2D animation...,Arts,short,230.0,http://www.outschool.com/classes/7a5b5c51-16b0...


,title,meeting_start_week,meeting_start_date,meeting_start_dow,meeting_start_hour,meeting_end_hour
0,Adventures in Crochet - Make a Potholder!,2020-07-13,2020-07-13,Mon,10:20AM,11:10AM
1,How To Make Awesome Animated Movies (Intro to ...,2020-07-13,2020-07-13,Mon,02:00PM,02:55PM
2,Tree of Life: Sharks and Rays,2020-07-13,2020-07-14,Tue,10:00AM,10:55AM
3,Famous People who were Productive during Quara...,2020-07-13,2020-07-14,Tue,03:00PM,03:40PM
4,Exploring Watercolor Paint Techniques,2020-07-13,2020-07-19,Sun,10:00AM,10:45AM
5,Adventures in Crochet - Make a Potholder!,2020-07-20,2020-07-20,Mon,10:20AM,11:10AM
6,How To Make Awesome Animated Movies (Intro to ...,2020-07-20,2020-07-20,Mon,02:00PM,02:55PM
7,Adventures in Crochet - Make a Potholder!,2020-07-27,2020-07-27,Mon,10:20AM,11:10AM
8,How To Make Awesome Animated Movies (Intro to ...,2020-07-27,2020-07-27,Mon,02:00PM,02:55PM
9,How To Make Awesome Animated Movies (Intro to ...,2020-08-03,2020-08-03,Mon,02:00PM,02:55PM


___________________________________________________________________________________________________________


Example schedule #2:


,title,summary,subject,class_type,price_dollars,link
0,How To Make Awesome Animated Movies (Intro to ...,This course is an introduction to 2D animation...,Arts,short,230.0,http://www.outschool.com/classes/7a5b5c51-16b0...


...

,title,meeting_start_week,meeting_start_date,meeting_start_dow,meeting_start_hour,meeting_end_hour
0,Create Andy Warhol's Zebra with a Sprinkling o...,2020-07-13,2020-07-13,Mon,11:30AM,12:30PM
1,How To Make Awesome Animated Movies (Intro to ...,2020-07-13,2020-07-13,Mon,03:00PM,03:55PM
2,Kidpreneur Design: Let's Promote Your Next Big...,2020-07-13,2020-07-14,Tue,11:30AM,12:10PM
3,How To Make Awesome Animated Movies (Intro to ...,2020-07-20,2020-07-20,Mon,03:00PM,03:55PM
4,Kidpreneur Design: Let's Promote Your Next Big...,2020-07-20,2020-07-21,Tue,11:30AM,12:10PM
5,Kidpreneur Design: Let's Promote Your Next Big...,2020-07-27,2020-07-28,Tue,11:30AM,12:10PM
6,How To Make Awesome Animated Movies (Intro to ...,2020-08-03,2020-08-03,Mon,03:00PM,03:55PM
7,Kidpreneur Design: Let's Promote Your Next Big...,2020-08-03,2020-08-04,Tue,11:30AM,12:10PM
8,How To Make Awesome Animated Movies (Intro to ...,2020-08-10,2020-08-10,Mon,03:00PM,03:55PM
9,How To Make Awesome Animated Movies (Intro to ...,2020-08-17,2020-08-17,Mon,03:00PM,03:55PM


___________________________________________________________________________________________________________


Example schedule #4:


,title,summary,subject,class_type,price_dollars,link
0,The Art of Drawing an Eye!,"In this one time course, I will be demonstrati...",Arts,one_time,13.0,http://www.outschool.com/classes/18e7b85f-8047...
1,Secret Ciphers for Stealthy Communications,Learn several different methods to communicate...,Social Studies,one_time,13.0,http://www.outschool.com/classes/cef0de7a-dd38...
2,Speak Up: Loud and Proud!,"In this one-time class, students will be intro...",English,one_time,10.0,http://www.outschool.com/classes/e45b0cc2-11c5...
3,Draw It: Sea Turtles,Learn to draw a scene with baby sea turtles!,Arts,one_time,14.0,http://www.outschool.com/classes/4ab6269c-6434...
4,Baking Class for Kids,Students will learn the art of baking from bas...,Arts,short,40.0,http://www.outschool.com/classes/bfcf61a4-aac2...
5,Paper Engineering 101-Let's get cutting! Creat...,"In this 45 minute class, you will learn how ju...",Arts,one_time,10.0,http://www.outschool.com/classes/c35d14be-e6ba...
6,Escape Room: Red Pandas Escape the Zoo!,This is a one time escape room class where you...,Science & Nature,one_time,14.0,http://www.outschool.com/classes/7aed559c-5d1b...
7,Introduction to Veterinary / Vet Medicine,Explore some of the things that veterinary pro...,Science & Nature,short,100.0,http://www.outschool.com/classes/f4030141-93e1...
8,Step by Step Art Creations: Kawaii Food Doodles,"Each class, I will walk my students through st...",Arts,one_time,13.0,http://www.outschool.com/classes/32e1b663-1b65...


,title,meeting_start_week,meeting_start_date,meeting_start_dow,meeting_start_hour,meeting_end_hour
0,Escape Room: Red Pandas Escape the Zoo!,2020-07-13,2020-07-13,Mon,10:00AM,10:55AM
1,Speak Up: Loud and Proud!,2020-07-13,2020-07-13,Mon,11:00AM,11:45AM
2,Baking Class for Kids,2020-07-13,2020-07-13,Mon,03:00PM,03:30PM
3,Draw It: Sea Turtles,2020-07-13,2020-07-14,Tue,10:00AM,10:45AM
4,Paper Engineering 101-Let's get cutting! Creat...,2020-07-13,2020-07-14,Tue,11:00AM,12:00PM
5,Secret Ciphers for Stealthy Communications,2020-07-13,2020-07-14,Tue,02:00PM,03:00PM
6,The Art of Drawing an Eye!,2020-07-13,2020-07-18,Sat,10:00AM,10:55AM
7,Step by Step Art Creations: Kawaii Food Doodles,2020-07-13,2020-07-18,Sat,11:30AM,12:25PM
8,Introduction to Veterinary / Vet Medicine,2020-07-13,2020-07-18,Sat,03:00PM,03:55PM
9,Baking Class for Kids,2020-07-20,2020-07-20,Mon,03:00PM,03:30PM


___________________________________________________________________________________________________________




In [88]:
df1 = pd.read_csv('query_v1.csv') ## https://outschool-metabase.herokuapp.com/question/6319
df2 = pd.read_csv('query_v2.csv') ## https://outschool-metabase.herokuapp.com/question/6320

In [89]:
learner_age = 10
subjects = set(['English', 'Social Studies', 'Arts', 'Science & Nature'])
class_types = set(['one_time', 'camp', 'short', 'semester', 'ongoing'])
budget = 100
hours_of_class = 30
start_date = '2020-07-12'
end_date = '2020-07-25'
availability_dow = set(['Mon', 'Tue', 'Sat', 'Sun'])
availability_hod = set([10, 11, 14, 15])
teacher_min_star_rating = 4.5

In [90]:
df1_f1 = df1[df1['subject'].isin(subjects)]
df1_f2 = df1_f1[(df1_f1['age_min'] <= learner_age) & (df1_f1['age_max'] >= learner_age)] 
df1_f3 = df1_f2[df1_f2['teacher_avg_star_rating'] >= teacher_min_star_rating]
df1_f4 = df1_f3[df1_f3['class_type'].isin(class_types)]

eligible_sections = []

for section_uid in df1_f4['section_uid'].unique():
    eligible = 'true'
    meeting_dow = df1_f4[df1_f4['section_uid'] == section_uid]['meeting_start_dow']
    meeting_hod = df1_f4[df1_f4['section_uid'] == section_uid]['meeting_start_hour_of_day']

    l1 = list(set(meeting_dow) - set(availability_dow))
    l2 = list(set(meeting_hod) - set(availability_hod))

    if len(l1) == 0 and len(l2) == 0: 
        eligible_sections.append(section_uid)

df2_f1 = df2[df2['subject'].isin(subjects)]
df2_f2 = df2_f1[(df2_f1['age_min'] <= learner_age) & (df2_f1['age_max'] >= learner_age)] 
df2_f3 = df2_f2[df2_f2['teacher_avg_star_rating'] >= teacher_min_star_rating]
df2_f4 = df2_f3[df2_f3['section_uid'].isin(eligible_sections)]
df2_f5 = df2_f4[df2_f4['class_type'].isin(class_types)]
   

In [91]:
df2_f5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 12 to 2056
Data columns (total 61 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uid                      101 non-null    object 
 1   title                    101 non-null    object 
 2   summary                  101 non-null    object 
 3   teacher_avg_star_rating  101 non-null    float64
 4   subject                  101 non-null    object 
 5   age_min                  101 non-null    int64  
 6   age_max                  101 non-null    int64  
 7   size_max                 101 non-null    int64  
 8   class_type               101 non-null    object 
 9   duration_weeks           101 non-null    int64  
 10  weekly_meetings          101 non-null    int64  
 11  duration_minutes         101 non-null    int64  
 12  total_duration_minutes   101 non-null    int64  
 13  price_dollars            101 non-null    float64
 14  section_uid             

In [102]:
spend = 0
purchased_sections = []
visited_indices = []

while spend <= budget or abs(budget - spend) < 20:
    print('\n')
    print('\n')
    print('\n')
    print('\n')
    print('\n')
    print('\n')
    
    random_index = randint(0, len(df2_f5) - 1)

    if random_index in visited_indices:
        print('Already visited!')
        pass 

    else:
        visited_indices.append(random_index)
        random_row = df2_f5.iloc[random_index]
        activity_title = random_row['title']
        section_uid = random_row['section_uid']
        print('Candidate: ', section_uid)
        price_dollars = random_row['price_dollars']


        if len(purchased_sections) == 0:
            print('Added section!')
            purchased_sections.append(section_uid)
            spend += price_dollars

        else: 
            total_schedule_conflicts = 0
            for prev_section in purchased_sections:

                section_1_meetings = df1_f4[df1_f4['section_uid'] == section_uid].sort_values(by=['meeting_start_time'])
                section_2_meetings = df1_f4[df1_f4['section_uid'] == prev_section].sort_values(by=['meeting_start_time'])
                
                display(section_1_meetings[relevant_columns])
                display(section_2_meetings[relevant_columns])
                print('\n')
                
                relevant_columns = ['title',
                                    'section_uid',
                                    'meeting_start_time',
                                    'meeting_start_dow',
                                    'meeting_start_hour_of_day',
                                    'meeting_end_time'
                                   ]

                schedule_conflicts = 0 

                for i in range(len(section_1_meetings)):
                    row = section_1_meetings.iloc[i]
                    s1_meeting_start_time = row['meeting_start_time']
                    s1_meeting_end_time = row['meeting_end_time']
                    s1_meeting_start_dow = row['meeting_start_dow']
                    s1_meeting_start_hod = row['meeting_start_hour_of_day']

                    for j in range(len(section_2_meetings)):
                        row = section_2_meetings.iloc[j]
                        s2_meeting_start_time = row['meeting_start_time']
                        s2_meeting_end_time = row['meeting_end_time']
                        s2_meeting_start_dow = row['meeting_start_dow']
                        s2_meeting_start_hod = row['meeting_start_hour_of_day']

                        if (s1_meeting_start_time < s2_meeting_start_time < s1_meeting_end_time) or (s1_meeting_start_time < s2_meeting_end_time < s1_meeting_end_time) or (s2_meeting_start_time < s1_meeting_start_time < s2_meeting_end_time) or (s2_meeting_start_time < s1_meeting_end_time < s2_meeting_end_time):
                            print('Mini schedule conflict!')
                            schedule_conflicts += 1

                total_schedule_conflicts += schedule_conflicts

            if total_schedule_conflicts > 0:
                print('Overall schedule conflict!')
                pass
            
            else: 
                print('Added section!')
                purchased_sections.append(section_uid)
                spend += price_dollars














Candidate:  6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d
Added section!












Candidate:  04da58ef-90b8-485d-b238-c914ddff0ed7


,title,section_uid,meeting_start_time,meeting_start_dow,meeting_start_hour_of_day,meeting_end_time
3775,"Matter: It's All Around (Part 1 in the ""Introd...",04da58ef-90b8-485d-b238-c914ddff0ed7,2020-07-14T11:20:00.000-07:00,Tue,11.0,2020-07-14T12:15:00.000-07:00


,title,section_uid,meeting_start_time,meeting_start_dow,meeting_start_hour_of_day,meeting_end_time
158,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-07-13T15:00:00.000-07:00,Mon,15.0,2020-07-13T15:55:00.000-07:00
157,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-07-20T15:00:00.000-07:00,Mon,15.0,2020-07-20T15:55:00.000-07:00
156,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-07-27T15:00:00.000-07:00,Mon,15.0,2020-07-27T15:55:00.000-07:00
155,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-08-03T15:00:00.000-07:00,Mon,15.0,2020-08-03T15:55:00.000-07:00
154,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-08-10T15:00:00.000-07:00,Mon,15.0,2020-08-10T15:55:00.000-07:00
153,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-08-17T15:00:00.000-07:00,Mon,15.0,2020-08-17T15:55:00.000-07:00
152,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-08-24T15:00:00.000-07:00,Mon,15.0,2020-08-24T15:55:00.000-07:00
151,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-08-31T15:00:00.000-07:00,Mon,15.0,2020-08-31T15:55:00.000-07:00




Added section!












Candidate:  bc7bc16a-167c-4a19-879b-593004441bda


,title,section_uid,meeting_start_time,meeting_start_dow,meeting_start_hour_of_day,meeting_end_time
4840,Watercolor Wave with Surfer,bc7bc16a-167c-4a19-879b-593004441bda,2020-07-13T11:00:00.000-07:00,Mon,11.0,2020-07-13T11:55:00.000-07:00


...

,title,section_uid,meeting_start_time,meeting_start_dow,meeting_start_hour_of_day,meeting_end_time
5113,Egyptian Mythology: Deities and Beasts,fb4da1c1-639f-443e-b729-a205128e2cf2,2020-07-14T15:00:00.000-07:00,Tue,15.0,2020-07-14T16:00:00.000-07:00


,title,section_uid,meeting_start_time,meeting_start_dow,meeting_start_hour_of_day,meeting_end_time
770,The Art of Drawing an Eye!,26c74fba-5d28-427b-8684-17732cae662b,2020-07-18T10:00:00.000-07:00,Sat,10.0,2020-07-18T10:55:00.000-07:00


,title,section_uid,meeting_start_time,meeting_start_dow,meeting_start_hour_of_day,meeting_end_time
5113,Egyptian Mythology: Deities and Beasts,fb4da1c1-639f-443e-b729-a205128e2cf2,2020-07-14T15:00:00.000-07:00,Tue,15.0,2020-07-14T16:00:00.000-07:00


,title,section_uid,meeting_start_time,meeting_start_dow,meeting_start_hour_of_day,meeting_end_time
5044,Getting To Know American Girl Kit & Daily Life...,e3300396-777c-4a5e-bf49-f6b65103a5d4,2020-07-13T10:00:00.000-07:00,Mon,10.0,2020-07-13T10:45:00.000-07:00




Added section!


In [103]:
purchased_sections

# a8e881ad-d76d-4380-b192-604b4d5a94ea
# f1732019-dd65-4f32-ac3d-3d2bd6af1e14

['6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d',
 '04da58ef-90b8-485d-b238-c914ddff0ed7',
 'bc7bc16a-167c-4a19-879b-593004441bda',
 '26c74fba-5d28-427b-8684-17732cae662b',
 'e3300396-777c-4a5e-bf49-f6b65103a5d4',
 'fb4da1c1-639f-443e-b729-a205128e2cf2']

In [104]:
class_list = df2_f5[df2_f5['section_uid'].isin(purchased_sections)].copy()
meeting_list = df1_f4[df1_f4['section_uid'].isin(purchased_sections)].sort_values(by='meeting_start_time').copy()

class_list_cols = ['uid', 'section_uid', 'title', 'summary', 'subject', 'price_dollars']
class_list_filtered = class_list[class_list_cols].copy()
class_list_filtered['link'] = 'http://www.outschool.com/classes/' + class_list_filtered['uid']

meeting_list_cols = ['title', 'meeting_start_time', 'meeting_start_date', 'meeting_start_dow', 'meeting_start_hour', 'meeting_end_hour']
meeting_list['meeting_start_date'] = meeting_list['meeting_start_time'].apply(lambda x: convert_to_date(x))
meeting_list['meeting_start_hour'] = meeting_list['meeting_start_time'].apply(lambda x: convert_to_hours(x))
meeting_list['meeting_end_hour']   = meeting_list['meeting_end_time'].apply(lambda x: convert_to_hours(x))
meeting_list_filtered = meeting_list[meeting_list_cols].copy()

print('\n')
print("Class list output")
display(class_list_filtered)

print('\n')
print("Meeting list output")
display(meeting_list_filtered)
    
# return class_list_filtered, meeting_list_filtered




Class list output


,uid,section_uid,title,summary,subject,price_dollars,link
122,888eea7e-4fec-438c-9513-4a3f3dda4fbb,04da58ef-90b8-485d-b238-c914ddff0ed7,"Matter: It's All Around (Part 1 in the ""Introd...","In this course, we will learn what matter is a...",Science & Nature,20.0,http://www.outschool.com/classes/888eea7e-4fec...
224,18e7b85f-8047-4244-9d62-dd698aa88560,26c74fba-5d28-427b-8684-17732cae662b,The Art of Drawing an Eye!,"In this one time course, I will be demonstrati...",Arts,13.0,http://www.outschool.com/classes/18e7b85f-8047...
421,bd841822-858b-4340-bd5b-0b60ce8689f2,fb4da1c1-639f-443e-b729-a205128e2cf2,Egyptian Mythology: Deities and Beasts,"A taste of the best and worst gods, goddesses,...",English,18.0,http://www.outschool.com/classes/bd841822-858b...
525,05d71289-8bc6-40c7-b306-71cb66dc01de,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,Back to the Future: A Tour of Ancient Civiliza...,"In this 8-week course, students will study var...",Social Studies,50.0,http://www.outschool.com/classes/05d71289-8bc6...
850,ba802c9f-96ee-4fc2-9a08-1dd6d4ce012c,e3300396-777c-4a5e-bf49-f6b65103a5d4,Getting To Know American Girl Kit & Daily Life...,"In this one time class, you will learn about w...",Social Studies,10.0,http://www.outschool.com/classes/ba802c9f-96ee...
865,b2597c43-4995-434a-8760-bd213fbee210,bc7bc16a-167c-4a19-879b-593004441bda,Watercolor Wave with Surfer,Students will complete a 1-session step-by-ste...,Arts,10.0,http://www.outschool.com/classes/b2597c43-4995...




Meeting list output


,title,meeting_start_time,meeting_start_date,meeting_start_dow,meeting_start_hour,meeting_end_hour
5044,Getting To Know American Girl Kit & Daily Life...,2020-07-13T10:00:00.000-07:00,2020-07-13,Mon,10:00AM,10:45AM
4840,Watercolor Wave with Surfer,2020-07-13T11:00:00.000-07:00,2020-07-13,Mon,11:00AM,11:55AM
158,Back to the Future: A Tour of Ancient Civiliza...,2020-07-13T15:00:00.000-07:00,2020-07-13,Mon,03:00PM,03:55PM
3775,"Matter: It's All Around (Part 1 in the ""Introd...",2020-07-14T11:20:00.000-07:00,2020-07-14,Tue,11:20AM,12:15PM
5113,Egyptian Mythology: Deities and Beasts,2020-07-14T15:00:00.000-07:00,2020-07-14,Tue,03:00PM,04:00PM
770,The Art of Drawing an Eye!,2020-07-18T10:00:00.000-07:00,2020-07-18,Sat,10:00AM,10:55AM
157,Back to the Future: A Tour of Ancient Civiliza...,2020-07-20T15:00:00.000-07:00,2020-07-20,Mon,03:00PM,03:55PM
156,Back to the Future: A Tour of Ancient Civiliza...,2020-07-27T15:00:00.000-07:00,2020-07-27,Mon,03:00PM,03:55PM
155,Back to the Future: A Tour of Ancient Civiliza...,2020-08-03T15:00:00.000-07:00,2020-08-03,Mon,03:00PM,03:55PM
154,Back to the Future: A Tour of Ancient Civiliza...,2020-08-10T15:00:00.000-07:00,2020-08-10,Mon,03:00PM,03:55PM


In [87]:
find_schedule_conflicts('9d36a8b4-0f0c-4c57-97e9-7b2989c11f7c', 
                          '69a8d99a-0e58-40fb-bec9-10b682d53fe0',
                           df1_f4)

1



Class list output


,uid,section_uid,title,summary,subject,price_dollars,link
33,77eed2a3-fd46-4231-9d1d-60157784292f,b48f6996-b172-49cb-be74-5a5ce4de9b48,Descendants Dance Workshop,"In this one-time dance workshop, students will...",Arts,16.0,http://www.outschool.com/classes/77eed2a3-fd46...
98,155b812b-e42b-4645-8261-8502a2186a59,67e0b078-89cf-470c-9da6-a22530ddaabe,"What It Takes to Be a Bird: Ostrich, Peregrine...","In this class, kids will learn the characteris...",Science & Nature,11.0,http://www.outschool.com/classes/155b812b-e42b...
114,1acbde2a-d8d1-4cbc-9281-fe2ab705a669,1ed79f45-9828-4f38-949d-af1d3645b211,Famous People who were Productive during Quara...,This one-time class will present and discuss t...,Social Studies,10.0,http://www.outschool.com/classes/1acbde2a-d8d1...
215,b4a6a564-3a25-4bbe-bd35-eedfaeeb1b19,f1732019-dd65-4f32-ac3d-3d2bd6af1e14,My Hero Academia and the Real Science of Quirks,"In this class, we will look at the various her...",Science & Nature,26.0,http://www.outschool.com/classes/b4a6a564-3a25...
224,18e7b85f-8047-4244-9d62-dd698aa88560,26c74fba-5d28-427b-8684-17732cae662b,The Art of Drawing an Eye!,"In this one time course, I will be demonstrati...",Arts,13.0,http://www.outschool.com/classes/18e7b85f-8047...
396,3d1300d1-9bda-4bd8-93b0-83b979ba9364,0d510c78-2083-41d7-be32-ca0b3ba5aa07,Paint & Sip: Rainbow Ice Cream Cone!,Students will be led step-by-step on how to pa...,Arts,18.0,http://www.outschool.com/classes/3d1300d1-9bda...
739,606d067c-32a7-46fb-b653-ab3418b42397,be24ed40-0697-42a6-b308-298d1891800f,Let's Make Polymer Clay Miniatures: Animal Cro...,"In this one-time, 60-minute class, we will lea...",Arts,12.0,http://www.outschool.com/classes/606d067c-32a7...
903,7a5b5c51-16b0-4868-956b-5754b9e79156,f49ea6a7-0fd7-4323-b582-445929650327,How To Make Awesome Animated Movies (Intro to ...,This course is an introduction to 2D animation...,Arts,230.0,http://www.outschool.com/classes/7a5b5c51-16b0...
995,a0f1a342-2b40-42b6-b83a-db61b4d496f0,f130f6f2-32f7-4e43-b118-f98dd612ccf5,Beezus and Ramona BOOK CLUB,Join this exciting book club!! Enjoy discussio...,English,60.0,http://www.outschool.com/classes/a0f1a342-2b40...
1559,00de224f-d98a-46c5-b831-a49db25e19ad,f8877ad2-d677-4452-95f8-25b3587e3ca2,Lets Talk and Play Animal Crossing and Visit O...,Come join me and other learners as we chat abo...,Arts,10.0,http://www.outschool.com/classes/00de224f-d98a...




Meeting list output


,title,meeting_start_time,meeting_start_date,meeting_start_dow,meeting_start_hour,meeting_end_hour
4935,My Hero Academia and the Real Science of Quirks,2020-07-13T10:00:00.000-07:00,2020-07-13,Mon,10:00AM,10:55AM
6230,Capitalization Rule Book II (2 of 2),2020-07-13T10:15:00.000-07:00,2020-07-13,Mon,10:15AM,10:45AM
5323,Singing Classes for Beginners. The Weekly Class.,2020-07-13T11:15:00.000-07:00,2020-07-13,Mon,11:15AM,12:00PM
22,Lets Talk and Play Animal Crossing and Visit O...,2020-07-13T14:00:00.000-07:00,2020-07-13,Mon,02:00PM,03:30PM
2657,Let's Make Polymer Clay Miniatures: Animal Cro...,2020-07-13T14:00:00.000-07:00,2020-07-13,Mon,02:00PM,03:00PM
6193,Harry Potter Trivia Night - N.E.W.T. Level,2020-07-13T14:00:00.000-07:00,2020-07-13,Mon,02:00PM,02:55PM
3409,How To Make Awesome Animated Movies (Intro to ...,2020-07-13T14:00:00.000-07:00,2020-07-13,Mon,02:00PM,02:55PM
1809,Paint & Sip: Rainbow Ice Cream Cone!,2020-07-14T10:30:00.000-07:00,2020-07-14,Tue,10:30AM,12:00PM
646,"What It Takes to Be a Bird: Ostrich, Peregrine...",2020-07-14T11:00:00.000-07:00,2020-07-14,Tue,11:00AM,11:40AM
4158,Draw & Color Cute Cartoon Panda and Koala Bear...,2020-07-14T11:00:00.000-07:00,2020-07-14,Tue,11:00AM,11:45AM


In [163]:
find_schedule_conflicts_with_printing('6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d', 
                                      '0b129788-7783-4df5-9960-5807349282c2', 
                                      df1_f4)



,title,section_uid,meeting_start_time,meeting_start_dow,meeting_start_hour_of_day,meeting_end_time
158,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-07-13T15:00:00.000-07:00,Mon,15.0,2020-07-13T15:55:00.000-07:00
157,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-07-20T15:00:00.000-07:00,Mon,15.0,2020-07-20T15:55:00.000-07:00
156,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-07-27T15:00:00.000-07:00,Mon,15.0,2020-07-27T15:55:00.000-07:00
155,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-08-03T15:00:00.000-07:00,Mon,15.0,2020-08-03T15:55:00.000-07:00
154,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-08-10T15:00:00.000-07:00,Mon,15.0,2020-08-10T15:55:00.000-07:00
153,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-08-17T15:00:00.000-07:00,Mon,15.0,2020-08-17T15:55:00.000-07:00
152,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-08-24T15:00:00.000-07:00,Mon,15.0,2020-08-24T15:55:00.000-07:00
151,Back to the Future: A Tour of Ancient Civiliza...,6e6a2c66-bfc9-43b2-a4fc-f91edc6a888d,2020-08-31T15:00:00.000-07:00,Mon,15.0,2020-08-31T15:55:00.000-07:00


,title,section_uid,meeting_start_time,meeting_start_dow,meeting_start_hour_of_day,meeting_end_time
5177,Baking Class for Kids,0b129788-7783-4df5-9960-5807349282c2,2020-07-13T15:00:00.000-07:00,Mon,15.0,2020-07-13T15:30:00.000-07:00
5176,Baking Class for Kids,0b129788-7783-4df5-9960-5807349282c2,2020-07-20T15:00:00.000-07:00,Mon,15.0,2020-07-20T15:30:00.000-07:00
5175,Baking Class for Kids,0b129788-7783-4df5-9960-5807349282c2,2020-07-27T15:00:00.000-07:00,Mon,15.0,2020-07-27T15:30:00.000-07:00
5174,Baking Class for Kids,0b129788-7783-4df5-9960-5807349282c2,2020-08-03T15:00:00.000-07:00,Mon,15.0,2020-08-03T15:30:00.000-07:00




SCHEDULE CONFLICT
Section 1 meeting:  2020-07-13T15:00:00.000-07:00  -  2020-07-13T15:55:00.000-07:00
Section 2 meeting:  2020-07-13T15:00:00.000-07:00  -  2020-07-13T15:30:00.000-07:00
SCHEDULE CONFLICT
Section 1 meeting:  2020-07-20T15:00:00.000-07:00  -  2020-07-20T15:55:00.000-07:00
Section 2 meeting:  2020-07-20T15:00:00.000-07:00  -  2020-07-20T15:30:00.000-07:00
SCHEDULE CONFLICT
Section 1 meeting:  2020-07-27T15:00:00.000-07:00  -  2020-07-27T15:55:00.000-07:00
Section 2 meeting:  2020-07-27T15:00:00.000-07:00  -  2020-07-27T15:30:00.000-07:00
SCHEDULE CONFLICT
Section 1 meeting:  2020-08-03T15:00:00.000-07:00  -  2020-08-03T15:55:00.000-07:00
Section 2 meeting:  2020-08-03T15:00:00.000-07:00  -  2020-08-03T15:30:00.000-07:00


4

In [160]:
df2_f5[df2_f5['section_uid'] == 'bfcf61a4-aac2-4ac4-82a3-bfa6b305991c']

,uid,title,summary,teacher_avg_star_rating,subject,age_min,age_max,size_max,class_type,duration_weeks,weekly_meetings,duration_minutes,total_duration_minutes,price_dollars,section_uid,start_date,start_dow,start_hour_of_day,start_time,end_time,total_enrollments,meeting_1_start_time,meeting_1_end_time,meeting_2_start_time,meeting_2_end_time,meeting_3_start_time,meeting_3_end_time,meeting_4_start_time,meeting_4_end_time,meeting_5_start_time,meeting_5_end_time,meeting_6_start_time,meeting_6_end_time,meeting_7_start_time,meeting_7_end_time,meeting_8_start_time,meeting_8_end_time,meeting_9_start_time,meeting_9_end_time,meeting_10_start_time,meeting_10_end_time,meeting_11_start_time,meeting_11_end_time,meeting_12_start_time,meeting_12_end_time,meeting_13_start_time,meeting_13_end_time,meeting_14_start_time,meeting_14_end_time,meeting_15_start_time,meeting_15_end_time,meeting_16_start_time,meeting_16_end_time,meeting_17_start_time,meeting_17_end_time,meeting_18_start_time,meeting_18_end_time,meeting_19_start_time,meeting_19_end_time,meeting_20_start_time,meeting_20_end_time


In [161]:
df2_f5.head()

,uid,title,summary,teacher_avg_star_rating,subject,age_min,age_max,size_max,class_type,duration_weeks,weekly_meetings,duration_minutes,total_duration_minutes,price_dollars,section_uid,start_date,start_dow,start_hour_of_day,start_time,end_time,total_enrollments,meeting_1_start_time,meeting_1_end_time,meeting_2_start_time,meeting_2_end_time,meeting_3_start_time,meeting_3_end_time,meeting_4_start_time,meeting_4_end_time,meeting_5_start_time,meeting_5_end_time,meeting_6_start_time,meeting_6_end_time,meeting_7_start_time,meeting_7_end_time,meeting_8_start_time,meeting_8_end_time,meeting_9_start_time,meeting_9_end_time,meeting_10_start_time,meeting_10_end_time,meeting_11_start_time,meeting_11_end_time,meeting_12_start_time,meeting_12_end_time,meeting_13_start_time,meeting_13_end_time,meeting_14_start_time,meeting_14_end_time,meeting_15_start_time,meeting_15_end_time,meeting_16_start_time,meeting_16_end_time,meeting_17_start_time,meeting_17_end_time,meeting_18_start_time,meeting_18_end_time,meeting_19_start_time,meeting_19_end_time,meeting_20_start_time,meeting_20_end_time
12,ed28509a-b3ca-41f6-b444-86e6d9f725c4,Greek Mythology's Mightiest Hero: Hercules vs....,"In this mythological adventure, students will ...",4.976190,English,9,12,6,one_time,1,1,55,55,22.0,f077d843-26d1-4877-860c-6a90180594f7,2020-07-14T00:00:00.000-07:00,Tue,10.0,2020-07-14T10:30:00.000-07:00,2020-07-14T11:25:00.000-07:00,1,2020-07-14T10:30:00.000-07:00,2020-07-14T11:25:00.000-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,77eed2a3-fd46-4231-9d1d-60157784292f,Descendants Dance Workshop,"In this one-time dance workshop, students will...",4.814815,Arts,8,11,9,one_time,1,1,50,50,16.0,b48f6996-b172-49cb-be74-5a5ce4de9b48,2020-07-18T00:00:00.000-07:00,Sat,11.0,2020-07-18T11:00:00.000-07:00,2020-07-18T11:55:00.000-07:00,1,2020-07-18T11:00:00.000-07:00,2020-07-18T11:55:00.000-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,0c45dd02-476b-4122-8f67-3c4a58a819a2,Drawing: Learn How to Draw a Reddish And Ripe ...,Students will learn how to sketch an apple wit...,4.722222,Arts,10,15,12,one_time,1,1,60,60,10.0,81495144-0791-4594-84e5-3b60e28d2552,2020-07-14T00:00:00.000-07:00,Tue,15.0,2020-07-14T15:00:00.000-07:00,2020-07-14T16:00:00.000-07:00,1,2020-07-14T15:00:00.000-07:00,2020-07-14T16:00:00.000-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,155b812b-e42b-4645-8261-8502a2186a59,"What It Takes to Be a Bird: Ostrich, Peregrine...","In this class, kids will learn the characteris...",4.884615,Science & Nature,6,10,9,one_time,1,1,45,45,11.0,67e0b078-89cf-470c-9da6-a22530ddaabe,2020-07-14T00:00:00.000-07:00,Tue,11.0,2020-07-14T11:00:00.000-07:00,2020-07-14T11:40:00.000-07:00,1,2020-07-14T11:00:00.000-07:00,2020-07-14T11:40:00.000-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,1acbde2a-d8d1-4cbc-9281-fe2ab705a669,Famous People who were Productive during Quara...,This one-time class will present and discuss t...,4.825397,Social Studies,7,11,10,one_time,1,1,40,40,10.0,1ed79f45-9828-4f38-949d-af1d3645b211,2020-07-14T00:00:00.000-07:00,Tue,15.0,2020-07-14T15:00:00.000-07:00,2020-07-14T15:40:00.000-07:00,1,2020-07-14T15:00:00.000-07:00,2020-07-14T15:40:00.000-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# include hyperlinked title to class
# add class type filter
# clean up outputs
# run multiple schedules (eg 3, 5)


# include visualized schedule
# add topic filter
# add flex classes


In [18]:
example_schedule.head()

,uid,title,summary,teacher_avg_star_rating,subject,age_min,age_max,size_max,class_type,duration_weeks,weekly_meetings,duration_minutes,total_duration_minutes,price_dollars,section_uid,start_date,start_dow,start_hour_of_day,start_time,end_time,total_enrollments,meeting_uid,meeting_start_time,meeting_end_time,meeting_start_dow,meeting_start_hour_of_day
5021,b971cb54-6eb1-4f81-8e5c-79738b733dc9,How to Draw a Dragon in Two Styles (Folk and K...,Fast and fun drawing lesson on drawing a fanta...,4.951662,Arts,7,11,6,one_time,1,1,30,30,10.0,f06e5d84-a883-4f22-9d26-60e2f34c090e,2020-07-13T00:00:00.000-07:00,Mon,10.0,2020-07-13T10:30:00.000-07:00,2020-07-13T11:00:00.000-07:00,1,e17c5eae-a1d6-4bc7-a712-a89201556e1e,2020-07-13T10:30:00.000-07:00,2020-07-13T11:00:00.000-07:00,Mon,10.0
5432,c9506b9d-39ef-4a81-b08f-d0a40541ce7d,Tik Tok Escape The Room,Students will practice reading skills like mai...,5.000000,English,9,12,8,one_time,1,1,40,40,10.0,f43b0e41-b03a-49ad-8d40-cdee3650bf51,2020-07-13T00:00:00.000-07:00,Mon,11.0,2020-07-13T11:00:00.000-07:00,2020-07-13T11:40:00.000-07:00,1,cadc155d-f039-444c-b279-6fc68d8a205b,2020-07-13T11:00:00.000-07:00,2020-07-13T11:40:00.000-07:00,Mon,11.0
3404,7a5b5c51-16b0-4868-956b-5754b9e79156,How To Make Awesome Animated Movies (Intro to ...,This course is an introduction to 2D animation...,4.811881,Arts,9,14,18,short,5,1,55,55,230.0,2a878ac0-494c-408a-a640-676086b550de,2020-07-13T00:00:00.000-07:00,Mon,15.0,2020-07-13T15:00:00.000-07:00,2020-08-17T15:55:00.000-07:00,3,5304752c-0136-44e8-b7e6-d1db4cffefa9,2020-07-13T15:00:00.000-07:00,2020-07-13T15:55:00.000-07:00,Mon,15.0
1353,2c972021-afa8-4107-aba9-e8ed01208246,All About Cats: Cats Around the World,Have you ever wondered where cats come from? L...,4.965517,Science & Nature,9,12,7,one_time,1,1,55,55,14.0,67c16dc5-3b9c-41d5-acb1-56dfe0c209fc,2020-07-14T00:00:00.000-07:00,Tue,10.0,2020-07-14T10:00:00.000-07:00,2020-07-14T10:55:00.000-07:00,1,399fee03-aeb2-4509-bd9b-1a5921f5abfd,2020-07-14T10:00:00.000-07:00,2020-07-14T10:55:00.000-07:00,Tue,10.0
2110,4ab6269c-6434-4a72-a579-3d27d76f09fb,Draw It: Sea Turtles,Learn to draw a scene with baby sea turtles!,4.895480,Arts,7,12,13,one_time,1,1,45,45,14.0,53b45f58-57e9-4b23-aafa-dab148da2c02,2020-07-14T00:00:00.000-07:00,Tue,10.0,2020-07-14T10:00:00.000-07:00,2020-07-14T10:45:00.000-07:00,2,18012270-6859-489c-8e30-b3c39763e5a6,2020-07-14T10:00:00.000-07:00,2020-07-14T10:45:00.000-07:00,Tue,10.0


In [62]:
relevant_columns = ['title', 'meeting_start_time', 'meeting_end_time', 'meeting_start_dow', 'meeting_start_hour_of_day']
filtered_example_schedule = example_schedule[relevant_columns].copy()
filtered_example_schedule['meeting_start_date'] = filtered_example_schedule['meeting_start_time'].apply(lambda x: convert_to_date(x))
filtered_example_schedule['meeting_start_hour'] = filtered_example_schedule['meeting_start_time'].apply(lambda x: convert_to_hours(x))
filtered_example_schedule['meeting_end_hour'] = filtered_example_schedule['meeting_end_time'].apply(lambda x: convert_to_hours(x))


filtered_example_schedule.head(100)

,title,meeting_start_time,meeting_end_time,meeting_start_dow,meeting_start_hour_of_day,meeting_start_date,meeting_start_hour,meeting_end_hour
5021,How to Draw a Dragon in Two Styles (Folk and K...,2020-07-13T10:30:00.000-07:00,2020-07-13T11:00:00.000-07:00,Mon,10.0,2020-07-13,10:30AM,11:00AM
5432,Tik Tok Escape The Room,2020-07-13T11:00:00.000-07:00,2020-07-13T11:40:00.000-07:00,Mon,11.0,2020-07-13,11:00AM,11:40AM
3404,How To Make Awesome Animated Movies (Intro to ...,2020-07-13T15:00:00.000-07:00,2020-07-13T15:55:00.000-07:00,Mon,15.0,2020-07-13,03:00PM,03:55PM
1353,All About Cats: Cats Around the World,2020-07-14T10:00:00.000-07:00,2020-07-14T10:55:00.000-07:00,Tue,10.0,2020-07-14,10:00AM,10:55AM
2110,Draw It: Sea Turtles,2020-07-14T10:00:00.000-07:00,2020-07-14T10:45:00.000-07:00,Tue,10.0,2020-07-14,10:00AM,10:45AM
4811,Create a Digital Scrapbook of 2020 So Far!,2020-07-14T10:00:00.000-07:00,2020-07-14T10:50:00.000-07:00,Tue,10.0,2020-07-14,10:00AM,10:50AM
4424,Beginner's Dramatic Defense Against the Dark A...,2020-07-14T10:00:00.000-07:00,2020-07-14T10:45:00.000-07:00,Tue,10.0,2020-07-14,10:00AM,10:45AM
3890,Let's Draw a Sloth on a Llama!,2020-07-14T10:00:00.000-07:00,2020-07-14T10:50:00.000-07:00,Tue,10.0,2020-07-14,10:00AM,10:50AM
646,"What It Takes to Be a Bird: Ostrich, Peregrine...",2020-07-14T11:00:00.000-07:00,2020-07-14T11:40:00.000-07:00,Tue,11.0,2020-07-14,11:00AM,11:40AM
2744,Introduction to Essay Writing: Middle School B...,2020-07-14T11:00:00.000-07:00,2020-07-14T11:45:00.000-07:00,Tue,11.0,2020-07-14,11:00AM,11:45AM


In [44]:
from datetime import datetime

# date_time_str = '18/09/19 01:55:19'
date_time_str = '2020-07-28T11:00:00.000-07:00'
date_time_obj = datetime.strptime(date_time_str[:19], '%Y-%m-%dT%H:%M:%S')
print("The type of the date is now", type(date_time_obj))

The type of the date is now <class 'datetime.datetime'>


In [53]:
print(date_time_obj.strftime("%B")) # month
print(date_time_obj.strftime("%Y-%m-%d")) # month day
print(date_time_obj.strftime("%a")) # day of week
print(date_time_obj.strftime("%I:%M%p")) # HH:MM am/pm

July
2020-07-28
Tue
11:00AM


In [60]:
def convert_to_hours(date_time_str):
    date_time_obj = datetime.strptime(date_time_str[:19], '%Y-%m-%dT%H:%M:%S')
    return date_time_obj.strftime("%I:%M%p")

def convert_to_date(date_time_str):
    date_time_obj = datetime.strptime(date_time_str[:19], '%Y-%m-%dT%H:%M:%S')
    return date_time_obj.strftime("%Y-%m-%d")


In [61]:
filtered_example_schedule['meeting_start_date'] = filtered_example_schedule['meeting_start_time'].apply(lambda x: convert_to_date(x))
filtered_example_schedule['meeting_start_hour'] = filtered_example_schedule['meeting_start_time'].apply(lambda x: convert_to_hours(x))
filtered_example_schedule['meeting_end_hour'] = filtered_example_schedule['meeting_end_time'].apply(lambda x: convert_to_hours(x))
filtered_example_schedule.head()

,title,meeting_start_time,meeting_end_time,meeting_start_dow,meeting_start_hour_of_day,meeting_start_week,meeting_start_hour,meeting_end_hour,meeting_start_date
5021,How to Draw a Dragon in Two Styles (Folk and K...,2020-07-13T10:30:00.000-07:00,2020-07-13T11:00:00.000-07:00,Mon,10.0,10:30AM,10:30AM,11:00AM,2020-07-13
5432,Tik Tok Escape The Room,2020-07-13T11:00:00.000-07:00,2020-07-13T11:40:00.000-07:00,Mon,11.0,11:00AM,11:00AM,11:40AM,2020-07-13
3404,How To Make Awesome Animated Movies (Intro to ...,2020-07-13T15:00:00.000-07:00,2020-07-13T15:55:00.000-07:00,Mon,15.0,03:00PM,03:00PM,03:55PM,2020-07-13
1353,All About Cats: Cats Around the World,2020-07-14T10:00:00.000-07:00,2020-07-14T10:55:00.000-07:00,Tue,10.0,10:00AM,10:00AM,10:55AM,2020-07-14
2110,Draw It: Sea Turtles,2020-07-14T10:00:00.000-07:00,2020-07-14T10:45:00.000-07:00,Tue,10.0,10:00AM,10:00AM,10:45AM,2020-07-14


In [130]:
def build_schedule_no_print(learner_age, subjects, class_types, budget, teacher_min_star_rating):
    
    df1_f1 = df1[df1['subject'].isin(subjects)]
    df1_f2 = df1_f1[(df1_f1['age_min'] <= learner_age) & (df1_f1['age_max'] >= learner_age)] 
    df1_f3 = df1_f2[df1_f2['teacher_avg_star_rating'] >= teacher_min_star_rating]
    df1_f4 = df1_f3[df1_f3['class_type'].isin(class_types)]
    
    eligible_sections = []

    for section_uid in df1_f4['section_uid'].unique():
        eligible = 'true'
        meeting_dow = df1_f3[df1_f3['section_uid'] == section_uid]['meeting_start_dow']
        meeting_hod = df1_f3[df1_f3['section_uid'] == section_uid]['meeting_start_hour_of_day']

        l1 = list(set(meeting_dow) - set(availability_dow))
        l2 = list(set(meeting_hod) - set(availability_hod))

        if len(l1) == 0 and len(l2) == 0: 
            eligible_sections.append(section_uid)
    
    df2_f1 = df2[df2['subject'].isin(subjects)]
    df2_f2 = df2_f1[(df2_f1['age_min'] <= learner_age) & (df2_f1['age_max'] >= learner_age)] 
    df2_f3 = df2_f2[df2_f2['teacher_avg_star_rating'] >= teacher_min_star_rating]
    df2_f4 = df2_f3[df2_f3['section_uid'].isin(eligible_sections)]
    df2_f5 = df2_f4[df2_f4['class_type'].isin(class_types)]
    
    spend = 0
    purchased_sections = []
    visited_indices = []
    
    while spend <= budget or abs(budget - spend) < 20:
        
        random_index = randint(0, len(df2_f4) - 1)
        
        if random_index in visited_indices:
            pass 
        
        else:
            visited_indices.append(random_index)
            random_row = df2_f5.iloc[random_index]
            activity_title = random_row['title']
            section_uid = random_row['section_uid']
            price_dollars = random_row['price_dollars']
            
            
            if len(purchased_sections) == 0:
                purchased_sections.append(section_uid)
                spend += price_dollars
            
            else: 
                schedule_conflicts = 0
                for prev_section in purchased_sections:
                    schedule_conflicts += find_schedule_conflicts(section_uid, prev_section)
                
                if schedule_conflicts > 0:
                    pass
                else: 
                    purchased_sections.append(section_uid)
                    spend += price_dollars
    
    class_list = df2_f5[df2_f5['section_uid'].isin(purchased_sections)].copy()
    meeting_list = df1_f4[df1_f4['section_uid'].isin(purchased_sections)].sort_values(by='meeting_start_time').copy()
    
    class_list_cols = ['uid', 'title', 'summary', 'subject', 'price_dollars']
    class_list_filtered = class_list[class_list_cols].copy()
    class_list_filtered['link'] = 'http://www.outschool.com/classes/' + class_list_filtered['uid']
    
    meeting_list_cols = ['title', 'meeting_start_time', 'meeting_start_date', 'meeting_start_dow', 'meeting_start_hour', 'meeting_end_hour']
    meeting_list['meeting_start_date'] = meeting_list['meeting_start_time'].apply(lambda x: convert_to_date(x))
    meeting_list['meeting_start_hour'] = meeting_list['meeting_start_time'].apply(lambda x: convert_to_hours(x))
    meeting_list['meeting_end_hour']   = meeting_list['meeting_end_time'].apply(lambda x: convert_to_hours(x))
    meeting_list_filtered = meeting_list[meeting_list_cols].copy()
    
    print('\n')
    print("Class list output")
    display(class_list_filtered)
    
    print('\n')
    print("Meeting list output")
    display(meeting_list_filtered)
    
    return class_list_filtered, meeting_list_filtered

In [131]:
learner_age = 10
subjects = set(['English', 'Social Studies', 'Arts', 'Science & Nature'])
class_types = set(['one_time', 'camp', 'short', 'semester', 'ongoing'])
budget = 500
hours_of_class = 30
start_date = '2020-07-12'
end_date = '2020-07-25'
availability_dow = set(['Mon', 'Tue', 'Sat', 'Sun'])
availability_hod = set([10, 11, 14, 15])
teacher_min_star_rating = 4.5 

# flex add in, topic filter, think about way to edit classes, make tool portable (end of day)


for i in range(1,2):
    print('\n')
    print('\n')
    print('\n')
    print('Example schedule #', i)
    build_schedule_no_print(learner_age, subjects, class_types, budget, teacher_min_star_rating)
    print('________________________________________________________________________________')







Example schedule # 1


Class list output


,uid,title,summary,subject,price_dollars,link
114,1acbde2a-d8d1-4cbc-9281-fe2ab705a669,Famous People who were Productive during Quara...,This one-time class will present and discuss t...,Social Studies,10.0,http://www.outschool.com/classes/1acbde2a-d8d1...
310,f9b9639e-af13-4c4b-9d3f-74c9ca3f3221,Dance Theater Camp!,"In this multi-day course, kids will gain confi...",Arts,48.0,http://www.outschool.com/classes/f9b9639e-af13...
326,a6ddf880-2e63-4018-b245-e7916167c77f,Creating Comics!,In this 4 week course students will learn the ...,Arts,73.0,http://www.outschool.com/classes/a6ddf880-2e63...
370,4467f390-5c5a-4fb6-9a32-c6733c2eef4c,Dragonology,Dare To Dance With Dragons And Learn Their Lore!,English,35.0,http://www.outschool.com/classes/4467f390-5c5a...
591,a11572c2-e724-4b1b-921a-e06ba1dff0a4,Beginner's Dramatic Defense Against the Dark A...,For kids who love the first Harry Potter book ...,Arts,11.0,http://www.outschool.com/classes/a11572c2-e724...
623,bedef3f5-ec36-4b53-86fe-4ba92a737955,Travel Tribe: Taking on Thailand - Buddhas & T...,Students will explore the most popular country...,Science & Nature,20.0,http://www.outschool.com/classes/bedef3f5-ec36...
634,bfcf61a4-aac2-4ac4-82a3-bfa6b305991c,Baking Class for Kids,Students will learn the art of baking from bas...,Arts,40.0,http://www.outschool.com/classes/bfcf61a4-aac2...
727,458c4607-45c7-415a-9347-ca980c799811,Toy Engineering: Marble Run (STEM Challenge),"In this STEM class, students will have the opp...",Science & Nature,12.0,http://www.outschool.com/classes/458c4607-45c7...
938,e79a9e5b-11e1-4bf2-bd77-8753bfc6c245,Around the World! The Seven Continents: Europe.,Join us as we explore the seven continents of ...,Science & Nature,10.0,http://www.outschool.com/classes/e79a9e5b-11e1...
953,193d8f53-8169-40cb-88ab-3bd67f15cedf,"Draw a Colorful, Impossible Triangle/Magical O...",In this one-time class students will learn how...,Arts,8.0,http://www.outschool.com/classes/193d8f53-8169...




Meeting list output


,title,meeting_start_time,meeting_start_date,meeting_start_dow,meeting_start_hour,meeting_end_hour
3762,Artistic Journaling for Joy,2020-07-13T10:00:00.000-07:00,2020-07-13,Mon,10:00AM,10:30AM
779,"Draw a Colorful, Impossible Triangle/Magical O...",2020-07-13T11:00:00.000-07:00,2020-07-13,Mon,11:00AM,11:25AM
5938,Dance like the “Descendants” to “What’s my Nam...,2020-07-13T11:00:00.000-07:00,2020-07-13,Mon,11:00AM,11:50AM
6193,Harry Potter Trivia Night - N.E.W.T. Level,2020-07-13T14:00:00.000-07:00,2020-07-13,Mon,02:00PM,02:55PM
1758,Let's Dance: Intermediate Tap,2020-07-13T14:00:00.000-07:00,2020-07-13,Mon,02:00PM,02:55PM
6632,Dance Theater Camp!,2020-07-13T15:00:00.000-07:00,2020-07-13,Mon,03:00PM,03:45PM
1929,Horses Around the World: Iceland,2020-07-13T15:00:00.000-07:00,2020-07-13,Mon,03:00PM,03:45PM
5498,Let's Explore the World: Europe!,2020-07-13T15:00:00.000-07:00,2020-07-13,Mon,03:00PM,04:00PM
5177,Baking Class for Kids,2020-07-13T15:00:00.000-07:00,2020-07-13,Mon,03:00PM,03:30PM
6042,There and Back Again: A Hobbit's Tale--Book St...,2020-07-14T10:00:00.000-07:00,2020-07-14,Tue,10:00AM,10:45AM


________________________________________________________________________________


In [128]:
def find_schedule_conflicts_with_printing(section_1, section_2, data):
    section_1_meetings = data[data['section_uid'] == section_1].sort_values(by=['meeting_start_time'])
    section_2_meetings = data[data['section_uid'] == section_2].sort_values(by=['meeting_start_time'])
    
    relevant_columns = ['title',
                        'section_uid',
                        'meeting_start_time',
                        'meeting_start_dow',
                        'meeting_start_hour_of_day',
                        'meeting_end_time'
                       ]
    display(section_1_meetings[relevant_columns])
    display(section_2_meetings[relevant_columns])
    print('\n')
    
    schedule_conflicts = 0 
    
    for i in range(len(section_1_meetings)):
        row = section_1_meetings.iloc[i]
        s1_meeting_start_time = row['meeting_start_time']
        s1_meeting_end_time = row['meeting_end_time']
        s1_meeting_start_dow = row['meeting_start_dow']
        s1_meeting_start_hod = row['meeting_start_hour_of_day']
        
        for j in range(len(section_2_meetings)):
            row = section_2_meetings.iloc[j]
            s2_meeting_start_time = row['meeting_start_time']
            s2_meeting_end_time = row['meeting_end_time']
            s2_meeting_start_dow = row['meeting_start_dow']
            s2_meeting_start_hod = row['meeting_start_hour_of_day']
            
            if (s1_meeting_start_time < s2_meeting_start_time < s1_meeting_end_time) or (s1_meeting_start_time < s2_meeting_end_time < s1_meeting_end_time):
                schedule_conflicts += 1
                print('SCHEDULE CONFLICT')
                print('Section 1 meeting: ', s1_meeting_start_time, ' - ', s1_meeting_end_time)
                print('Section 2 meeting: ', s2_meeting_start_time, ' - ', s2_meeting_end_time)
    
    return schedule_conflicts
    

In [121]:
find_schedule_conflicts_with_printing('193d8f53-8169-40cb-88ab-3bd67f15cedf', '5fcf9d18-e441-4d4e-bb00-63a42d08d5a9', df1_f4)



,title,section_uid,meeting_start_time,meeting_start_dow,meeting_start_hour_of_day,meeting_end_time


,title,section_uid,meeting_start_time,meeting_start_dow,meeting_start_hour_of_day,meeting_end_time


0

In [132]:
df1_f4[df1_f4['section_uid'] == '8787b72c-32e2-4e8d-956f-a156120e0479']

,uid,title,summary,teacher_avg_star_rating,subject,age_min,age_max,size_max,class_type,duration_weeks,weekly_meetings,duration_minutes,total_duration_minutes,price_dollars,section_uid,start_date,start_dow,start_hour_of_day,start_time,end_time,total_enrollments,meeting_uid,meeting_start_time,meeting_end_time,meeting_start_dow,meeting_start_hour_of_day


In [ ]:
learner_age = 10
subjects = set(['English', 'Social Studies', 'Arts', 'Science & Nature'])
budget = 500
hours_of_class = 30
start_date = '2020-07-12'
end_date = '2020-07-25'
availability_dow = set(['Mon', 'Tue', 'Sat', 'Sun'])
availability_hod = set([10, 11, 14, 15])
teacher_min_star_rating = 4.5


df1.head(100)

In [ ]:
df1_f1 = df1[df1['subject'].isin(subjects)]
df1_f2 = df1_f1[(df1_f1['age_min'] <= learner_age) & (df1_f1['age_max'] >= learner_age)] 
df1_f3 = df1_f2[df1_f2['teacher_avg_star_rating'] >= 4.5]


In [ ]:
df1_f3.head()



In [ ]:
eligible_sections = []

for section_uid in df1['section_uid'].unique():
    eligible = 'true'
    meeting_dow = df1_f3[df1_f3['section_uid'] == section_uid]['meeting_start_dow']
    meeting_hod = df1_f3[df1_f3['section_uid'] == section_uid]['meeting_start_hour_of_day']
    
    l1 = list(set(meeting_dow) - set(availability_dow))
    l2 = list(set(meeting_hod) - set(availability_hod))
    
    if len(l1) == 0 and len(l2) == 0: 
        eligible_sections.append(section_uid)
        

In [ ]:
df2_f1 = df2[df2['subject'].isin(subjects)]
df2_f2 = df2_f1[(df2_f1['age_min'] <= learner_age) & (df2_f1['age_max'] >= learner_age)] 
df2_f3 = df2_f2[df2_f2['teacher_avg_star_rating'] >= 4.5]
df2_f4 = df2_f3[df2_f3['section_uid'].isin(eligible_sections)]
df2_f4.head(20)


In [ ]:
def find_schedule_conflicts(section_1, section_2, data):
    section_1_meetings = data[data['section_uid'] == section_1].sort_values(by=['meeting_start_time'])
    section_2_meetings = data[data['section_uid'] == section_2].sort_values(by=['meeting_start_time'])
    
    relevant_columns = ['title',
                        'section_uid',
                        'meeting_start_time',
                        'meeting_start_dow',
                        'meeting_start_hour_of_day',
                        'meeting_end_time'
                       ]
#     display(section_1_meetings[relevant_columns])
#     display(section_2_meetings[relevant_columns])
#     print('\n')
    
    schedule_conflicts = 0 
    
    for i in range(len(section_1_meetings)):
        row = section_1_meetings.iloc[i]
        s1_meeting_start_time = row['meeting_start_time']
        s1_meeting_end_time = row['meeting_end_time']
        s1_meeting_start_dow = row['meeting_start_dow']
        s1_meeting_start_hod = row['meeting_start_hour_of_day']
        
        for j in range(len(section_2_meetings)):
            row = section_2_meetings.iloc[j]
            s2_meeting_start_time = row['meeting_start_time']
            s2_meeting_end_time = row['meeting_end_time']
            s2_meeting_start_dow = row['meeting_start_dow']
            s2_meeting_start_hod = row['meeting_start_hour_of_day']
            
            if (s1_meeting_start_time < s2_meeting_start_time < s1_meeting_end_time) or (s1_meeting_start_time < s2_meeting_end_time < s1_meeting_end_time):
                schedule_conflicts += 1
#                 print('SCHEDULE CONFLICT')
#                 print('Section 1 meeting: ', s1_meeting_start_time, ' - ', s1_meeting_end_time)
#                 print('Section 2 meeting: ', s2_meeting_start_time, ' - ', s2_meeting_end_time)
    
    return schedule_conflicts
    

In [ ]:
# test two sections that shouldn't have conflict
find_schedule_conflicts('4bac0c14-526a-4a0f-b0c6-34b7ed155031', 'a46cd1bf-0062-4716-a8ae-136b5fa61434', df1_f3)


In [ ]:
# test two sections that should have conflict
find_schedule_conflicts('4bac0c14-526a-4a0f-b0c6-34b7ed155031', '42a9c8d5-b157-4f36-a7d5-11fe94df3a57')


In [ ]:
def build_schedule_with_printing(learner_age, subjects, class_types, budget, teacher_min_star_rating):
    
    print('Requirements')
    print('Learner age: ', learner_age)
    print('Subjects: ', subjects)
    print('Class types: ', class_types)
    print('Budget: ', budget)
    print('Teacher min star rating: ', teacher_min_star_rating)
    print('\n')
    
    df1_f1 = df1[df1['subject'].isin(subjects)]
    df1_f2 = df1_f1[(df1_f1['age_min'] <= learner_age) & (df1_f1['age_max'] >= learner_age)] 
    df1_f3 = df1_f2[df1_f2['teacher_avg_star_rating'] >= teacher_min_star_rating]
    df1_f4 = df1_f3[df1_f3['class_type'].isin(class_types)]
    
    eligible_sections = []

    for section_uid in df1['section_uid'].unique():
        eligible = 'true'
        meeting_dow = df1_f3[df1_f3['section_uid'] == section_uid]['meeting_start_dow']
        meeting_hod = df1_f3[df1_f3['section_uid'] == section_uid]['meeting_start_hour_of_day']

        l1 = list(set(meeting_dow) - set(availability_dow))
        l2 = list(set(meeting_hod) - set(availability_hod))

        if len(l1) == 0 and len(l2) == 0: 
            eligible_sections.append(section_uid)
    
    df2_f1 = df2[df2['subject'].isin(subjects)]
    df2_f2 = df2_f1[(df2_f1['age_min'] <= learner_age) & (df2_f1['age_max'] >= learner_age)] 
    df2_f3 = df2_f2[df2_f2['teacher_avg_star_rating'] >= teacher_min_star_rating]
    df2_f4 = df2_f3[df2_f3['section_uid'].isin(eligible_sections)]
    df2_f5 = df2_f4[df2_f4['class_type'].isin(class_types)]
    
    print('Eligible sections: ', len(df2_f4))
    print('\n')
    
    spend = 0
    purchased_sections = []
    visited_indices = []
    
    while spend <= budget or abs(budget - spend) < 20:
        print('\n')
        print('Spend: ', spend)
        print('Budget: ', budget)
        
        random_index = randint(0, len(df2_f4) - 1)
        
        if random_index in visited_indices:
            print('Already tried this!')
            pass 
        
        else:
            visited_indices.append(random_index)
            random_row = df2_f5.iloc[random_index]
            activity_title = random_row['title']
            section_uid = random_row['section_uid']
            price_dollars = random_row['price_dollars']
            
            
            if len(purchased_sections) == 0:
                purchased_sections.append(section_uid)
                print('Activity: ', activity_title)
                print('Section: ', section_uid)
                print('Price ($): ', price_dollars)
                spend += price_dollars
            
            else: 
                schedule_conflicts = 0
                for prev_section in purchased_sections:
                    schedule_conflicts += find_schedule_conflicts_with_printing(section_uid, prev_section, df1_f4)
                
                if schedule_conflicts > 0:
                    print('Schedule conflicts found!')
                    pass
                else: 
                    purchased_sections.append(section_uid)
                    print('Activity: ', activity_title)
                    print('Section: ', section_uid)
                    print('Price ($): ', price_dollars)
                    spend += price_dollars
    
    class_list = df2_f5[df2_f5['section_uid'].isin(purchased_sections)].copy()
    meeting_list = df1_f4[df1_f4['section_uid'].isin(purchased_sections)].sort_values(by='meeting_start_time').copy()
    
    class_list_cols = ['uid', 'title', 'summary', 'subject', 'price_dollars']
    class_list_filtered = class_list[class_list_cols].copy()
    class_list_filtered['link'] = 'http://www.outschool.com/classes/' + class_list_filtered['uid']
    
    meeting_list_cols = ['title', 'meeting_start_time', 'meeting_start_date', 'meeting_start_dow', 'meeting_start_hour', 'meeting_end_hour']
    meeting_list['meeting_start_date'] = meeting_list['meeting_start_time'].apply(lambda x: convert_to_date(x))
    meeting_list['meeting_start_hour'] = meeting_list['meeting_start_time'].apply(lambda x: convert_to_hours(x))
    meeting_list['meeting_end_hour']   = meeting_list['meeting_end_time'].apply(lambda x: convert_to_hours(x))
    meeting_list_filtered = meeting_list[meeting_list_cols].copy()
    
    print('\n')
    print("Class list output")
    display(class_list_filtered)
    
    print('\n')
    print("Meeting list output")
    display(meeting_list_filtered)
    
    return class_list_filtered, meeting_list_filtered
            
